In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Häufig verwendete Parameter für die Transpilation

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Paketversionen</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen die Verwendung dieser oder neuerer Versionen.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
Diese Seite beschreibt einige der häufiger verwendeten Parameter für die lokale Transpilation. Diese Parameter werden über Argumente für [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.generate_preset_pass_manager#qiskit.transpiler.generate_preset_pass_manager) oder [`transpile`](https://docs.quantum.ibm.com/api/qiskit/compiler#qiskit.compiler.transpile) konfiguriert.

<span id="approx-degree"></span>
## Approximationsgrad
Sie können den Approximationsgrad verwenden, um anzugeben, wie genau der resultierende Schaltkreis mit dem gewünschten (Eingabe-)Schaltkreis übereinstimmen soll. Dies ist eine Gleitkommazahl im Bereich (0.0 - 1.0), wobei 0.0 maximale Approximation und 1.0 (Standard) keine Approximation bedeutet. Kleinere Werte tauschen Ausgabegenauigkeit gegen einfachere Ausführung (d.h. weniger Gates).

Bei der Zwei-Qubit-Unitär-Synthese (verwendet in den Anfangsstufen aller Level und für die Optimierungsstufe mit Optimierungslevel 3) gibt dieser Wert die Ziel-Fidelity der Ausgabezerlegung an. Das heißt, wie viel Fehler eingeführt wird, wenn eine Matrixdarstellung eines Schaltkreises in diskrete Gates umgewandelt wird. Wenn der Approximationsgrad niedriger ist (mehr Approximation), wird sich der Ausgabeschaltkreis der Synthese stärker von der Eingabematrix unterscheiden, aber wahrscheinlich auch weniger Gates haben (da jede beliebige Zwei-Qubit-Operation perfekt mit höchstens drei CX-Gates zerlegt werden kann) und ist einfacher auszuführen.

Wenn der Approximationsgrad kleiner als 1.0 ist, können Schaltkreise mit einem oder zwei CX-Gates synthetisiert werden, was zu weniger Fehlern von der Hardware, aber mehr von der Approximation führt. Da CX das teuerste Gate in Bezug auf Fehler ist, kann es vorteilhaft sein, deren Anzahl auf Kosten der Fidelity in der Synthese zu verringern (diese Technik wurde verwendet, um das Quantenvolumen auf IBM®-Geräten zu erhöhen: [Validating quantum computers using randomized model circuits](https://arxiv.org/abs/1811.12926)).

Als Beispiel erzeugen wir ein zufälliges Zwei-Qubit-`UnitaryGate`, das in der Anfangsstufe synthetisiert wird. Das Setzen von `approximation_degree` kleiner als 1.0 kann einen approximativen Schaltkreis erzeugen. Wir müssen auch die `basis_gates` angeben, damit die Synthesemethode weiß, welche Gates sie für die approximative Synthese verwenden kann.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import random_unitary
from qiskit.transpiler import generate_preset_pass_manager

UU = random_unitary(4, seed=12345)
rand_U = UnitaryGate(UU)

qubits = QuantumRegister(2, name="q")
qc = QuantumCircuit(qubits)
qc.append(rand_U, qubits)
pass_manager = generate_preset_pass_manager(
    optimization_level=1,
    approximation_degree=0.85,
    basis_gates=["sx", "rz", "cx"],
)
approx_qc = pass_manager.run(qc)
print(approx_qc.count_ops()["cx"])

2


This yields an output of `2` because the approximation requires fewer CX gates.

<span id="seed"></span>
## Random number generator seed

Some parts of the transpiler are stochastic, so repeated transpilation runs may return different results. To obtain a reproducible result, you can set the seed for the pseudorandom number generator using the `seed_transpiler` argument. Repeated runs using the same seed will return the same results.

Example:

In [2]:
pass_manager = generate_preset_pass_manager(
    optimization_level=1, seed_transpiler=11, basis_gates=["sx", "rz", "cx"]
)
optimized_1 = pass_manager.run(qc)
optimized_1.draw("mpl")

<Image src="../docs/images/guides/common-parameters/extracted-outputs/dbc652e8-53a4-47a9-a66e-d9c1e5ef07c9-0.svg" alt="Output of the previous code cell" />

Dies ergibt eine Ausgabe von `2`, da die Approximation weniger CX-Gates erfordert.

<span id="seed"></span>
## Zufallszahlengenerator-Seed
Einige Teile des Transpilers sind stochastisch, sodass wiederholte Transpilationsläufe unterschiedliche Ergebnisse liefern können. Um ein reproduzierbares Ergebnis zu erhalten, können Sie den Seed für den Pseudozufallszahlengenerator mit dem Argument `seed_transpiler` festlegen. Wiederholte Läufe mit demselben Seed liefern dieselben Ergebnisse.

Beispiel:

In [3]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit.transpiler import Layout

backend = FakeSherbrooke()

a, b = qubits
initial_layout = Layout({a: 5, b: 6})

pass_manager = generate_preset_pass_manager(
    optimization_level=1, backend=backend, initial_layout=initial_layout
)
transpiled_circ = pass_manager.run(qc)

transpiled_circ.draw("mpl", idle_wires=False)

<Image src="../docs/images/guides/common-parameters/extracted-outputs/e18c034c-eb26-4d9d-81d7-37e0eafa17c7-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/common-parameters/extracted-outputs/dbc652e8-53a4-47a9-a66e-d9c1e5ef07c9-0.svg)

<span id="init-layout"></span>
## Initial Layout

Vor der Transpilation sind die Qubits in Ihrem Schaltkreis virtuelle Qubits, die nicht notwendigerweise physischen Qubits auf dem Ziel-Backend entsprechen. Sie können die anfängliche Zuordnung virtueller Qubits zu physischen Qubits mit dem Argument `initial_layout` angeben. Beachten Sie, dass sich das endgültige Qubit-Layout vom anfänglichen Layout unterscheiden kann, da der Transpiler Qubits durch Swap-Gates oder andere Mittel permutieren kann.

Im folgenden Beispiel konstruieren wir ein Initial Layout für das Mock-Backend [`FakeSherbrooke`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/fake-provider-fake-sherbrooke#fakesherbrooke), indem wir ein [`Layout`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.Layout)-Objekt erstellen. Unser Layout ordnet das erste Qubit unseres Schaltkreises dem Qubit 5 von Sherbrooke zu, und es ordnet das zweite Qubit unseres Schaltkreises dem Qubit 6 von Sherbrooke zu. Beachten Sie, dass physische Qubits immer durch ganze Zahlen dargestellt werden.

In [4]:
initial_layout = [5, 6]

pass_manager = generate_preset_pass_manager(
    optimization_level=1, backend=backend, initial_layout=initial_layout
)
transpiled_circ = pass_manager.run(qc)

transpiled_circ.draw("mpl", idle_wires=False)

<Image src="../docs/images/guides/common-parameters/extracted-outputs/a7800d8a-7354-48e4-a55f-f902ae28c875-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/common-parameters/extracted-outputs/e18c034c-eb26-4d9d-81d7-37e0eafa17c7-0.svg)

Zusätzlich zur Angabe eines Layout-Objekts können Sie auch eine Liste von ganzen Zahlen übergeben, wobei das $i$-te Element der Liste das physische Qubit enthält, dem das $i$-te Qubit zugeordnet werden soll. Beispiel:

In [5]:
from qiskit.visualization import plot_error_map

plot_error_map(backend, figsize=(30, 24))

<Image src="../docs/images/guides/common-parameters/extracted-outputs/8df57c6a-1ff4-4d58-9b7e-4378452c3025-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/common-parameters/extracted-outputs/a7800d8a-7354-48e4-a55f-f902ae28c875-0.svg)

Sie können die Funktion [`plot_error_map`](https://docs.quantum.ibm.com/api/qiskit/qiskit.visualization.plot_error_map) verwenden, um ein Diagramm des Geräte-Graphen mit Fehlerinformationen und mit den beschrifteten physischen Qubits zu erzeugen. Sie können auch ähnliche Diagramme auf der Seite [Compute resources](https://quantum.cloud.ibm.com/computers) anzeigen.